## Topics

- Runnable
- Prompt Template
- Intro to LangGraph and memory
- Output parsing

In [ ]:
# %pip install langchain langchain-openai
# !pip --version
# !pip install grandalf

In [ ]:
# MAKE SURE LANGCHAIN VERSION IS 0.2.17

import langchain
print(langchain.__version__)

### Configurazione del Modello LLM con LangChain e OpenAI
Possiamo con langchain e i suoi package configurare un modello di linguaggio (LLM) utilizzando la libreria `langchain_openai` per interagire con i servizi di OpenAI. L'oggetto `ChatOpenAI` consente di impostare i parametri del modello, come la chiave API, la temperatura per controllare la creatività delle risposte, il numero massimo di token e il timeout della richiesta.

### Assignment per i Corsisti
- Scrivi il codice per creare un'istanza di `ChatOpenAI` con i parametri forniti.
- Assicurati di inserire una chiave API valida al posto di `"HERE YOUR API KEY"`.
- Come esercizio, prova a modificare i valori di `temperature` e `max_tokens` e osserva come queste modifiche influenzano le risposte del modello.
- Invoca il modello chiamando il metodo "invoke"

### Creazione di un Prompt Template e Collegamento al Modello
Si utilizza `ChatPromptTemplate` per creare un template di prompt personalizzato che guida le risposte del modello LLM. In questo esempio, il prompt include un messaggio di sistema che imposta il contesto, chiedendo al modello di rispondere come un ingegnere di Machine Learning di livello mondiale, e di concludere ogni risposta con un riferimento alla bellezza dell'uso della data science nelle decisioni. Il template accetta un input dell'utente tramite un segnaposto `{input}`. Infine, il prompt viene concatenato al modello LLM per creare una catena eseguibile.

### Assignment per i Corsisti
- Scrivi il codice per creare un `ChatPromptTemplate` con messaggi personalizzati.
- Concatenalo al modello LLM per eseguire la catena di prompt.
- Come esercizio, modifica il messaggio di sistema per esplorare come le risposte del modello cambiano in base alle variazioni del contesto fornito.


In [2]:
# from langchain.prompts import ChatPromptTemplate  # pip install langchain

## Interfaccia Runnable

Per semplificare la creazione di catene di eventi/esecuzioni anche molto complesse, tutti i componenti di LangChain implementano un protocollo "runnable" attraverso un'interfaccia comune che consente l'utilizzo standard di qualsiasi componente. Di seguito sono riportati i tre metodi principali:

* **stream** - invia risposte parziali man mano che vengono generate
* **invoke** - esegue la catena su un singolo input
* **batch** - esegue la catena su più input

### Input e Output dei Componenti Principali
<img src="assets/componenti_io.png" width="600">

Uno dei vantaggi delle interfacce Runnable è che i componenti eseguibili possono essere collegati insieme in sequenze di esecuzione, permettendo all'output di un componente di diventare automaticamente l'input di un altro. Il comando *pipe* **|** è utilizzato a questo scopo nel LCEL (LangChain Expression Language), consentendo la creazione di componenti eseguibili da altri componenti eseguibili configurandoli in una sequenza che lavorerà in modo sinergico.


### Invocazione della Catena con un Input
In questa cella, si esegue la catena creata passando un input dell'utente al modello. Il metodo `invoke` permette di inviare un messaggio alla catena e ricevere la risposta generata dal modello, seguendo il contesto e le istruzioni definite nel `ChatPromptTemplate`.

### Assignment per i Corsisti
- Scrivi il codice per invocare la catena con un input personalizzato e osserva la risposta generata.
- Analizza la risposta del modello per verificare se segue le istruzioni del prompt, come terminare con un riferimento alla data science.
- Come esercizio, prova a invocare la catena con altri input e osserva come il contesto influisce sulle risposte.


In [ ]:
# chain.get_graph().print_ascii()

### Flusso di Lavoro per Conversazioni AI con LangChain e LangGraph

Questo codice definisce un flusso di lavoro per gestire conversazioni AI utilizzando le librerie **LangChain** e **LangGraph**.

#### Creazione del Grafo di Stato
Si crea un oggetto `StateGraph` basato su uno schema di stato dei messaggi, che rappresenta il flusso di esecuzione:

```python
workflow = StateGraph(state_schema=MessagesState)
def call_model(state: MessagesState):
    ...

```
#### Funzione per Chiamare il Modello
La funzione `call_model`  prepara un prompt iniziale e aggiunge i messaggi provenienti dallo stato. Questi messaggi vengono poi passati al modello per ottenere una risposta.

#### Aggiunta di Nodi e Collegamenti
Viene aggiunto un nodo chiamato "model" al flusso di lavoro e viene creato un collegamento di esecuzione dal nodo di inizio (`START`) al nodo "model"

#### Checkpointer in Memoria
Per salvare e recuperare lo stato della conversazione, viene aggiunto un oggetto `MemorySaver` come `checkpointer`

#### Compilazione del Flusso di Lavoro
Il flusso di lavoro viene compilato per diventare eseguibile. 

Questo approccio consente di creare flussi di lavoro modulari per la gestione delle conversazioni AI, con nodi eseguibili e collegamenti che facilitano la gestione dei messaggi e la memoria delle sessioni.

In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph



### Invocazione del Flusso di Lavoro con Input Personalizzato
Questa cella invoca l'applicazione del flusso di lavoro precedentemente compilato, passando un messaggio specifico come input. In questo esempio, il messaggio richiede al modello di tradurre la frase "I love programming" in francese. Inoltre, la configurazione include un `thread_id` per tracciare l'esecuzione in un contesto specifico.

### Assignment per i Corsisti
- Scrivi il codice per invocare l'applicazione con un input e un contesto configurabile.
- Osserva la risposta generata per verificare la correttezza della traduzione e l'aderenza alle istruzioni del prompt.
- Come esercizio, prova a modificare l'input e testare diverse frasi o richieste per vedere come il modello gestisce le variazioni.


In [ ]:
resp['messages'][-1].content

In [4]:
# Play around and test memory or change threads

In [5]:

#for r in resp['messages']:
#    print(type(r).__name__, r.content)
#    print("\n")

# Introduzione a PromptTemplate

Il `PromptTemplate` è una funzionalità potente progettata per semplificare e standardizzare la creazione di prompt per varie applicazioni, come chatbot, risposte automatiche o moduli di inserimento dati. Fornisce un formato strutturato che può essere riutilizzato in diversi scenari, garantendo coerenza ed efficienza nel modo in cui vengono richiesti e gestiti gli input.


In [ ]:
# dynamic template and use of a Memory Buffer

template = """Act as a data scientist answering to every question with references to the beauty of Data Science.
New question: {question}
Answer:"""

from langchain_core.prompts.prompt import PromptTemplate

prompt = PromptTemplate(template=template)


from langchain.chains import LLMChain

conversation = prompt | llm 

In [ ]:
conversation.invoke({"question": "Hello, i like the orange color."})

## Parsing dell'Output degli LLM

<a href="https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html" target="_blank">fonte</a>

I modelli di linguaggio (LLM) generano testo. Tuttavia, spesso è necessario ottenere informazioni più strutturate rispetto al semplice testo grezzo. È qui che entrano in gioco i parser di output.

**I parser di output** sono classi che aiutano a *strutturare le risposte dei modelli di linguaggio*.

Un parser di output deve implementare principalmente due metodi:

- **"Get format instructions"**: Un metodo che restituisce una stringa contenente le istruzioni su come deve essere formattato l'output di un modello di linguaggio.
- **"Parse"**: Un metodo che accetta una stringa (presumibilmente la risposta di un modello di linguaggio) e la analizza in una struttura.

Esiste anche un metodo opzionale:

- **"Parse with prompt"**: Un metodo che accetta una stringa (presumibilmente la risposta di un modello di linguaggio) e un prompt (presumibilmente il prompt che ha generato tale risposta) e la analizza in una struttura. Il prompt viene fornito principalmente nel caso in cui il parser di output voglia riprovare o correggere l'output in qualche modo, utilizzando le informazioni del prompt per farlo.


### Assignment: Utilizzo di OutputParser con un Modello LLM
In questa cella, si utilizza `OutputParser` per formattare e strutturare le risposte generate da un modello di linguaggio. Si crea un template di prompt che istruisce il modello a rispondere come un data scientist, includendo riferimenti alla bellezza della Data Science. Si utilizza poi `StrOutputParser` per elaborare e formattare l'output del modello.

### Assignment per i Corsisti
- Scrivi il codice per creare un `PromptTemplate` con un template personalizzato e concatenalo al modello LLM.
- Configura l'`OutputParser` per strutturare l'output del modello.
- Esegui la catena di esecuzione con un input di esempio e verifica la formattazione della risposta.
- Come esercizio, modifica il template per includere altri dettagli specifici e osserva come cambia la risposta del modello.


In [6]:
# StrOutputParser

### Parsing dell'Output con Pydantic e LangChain
In questa cella, viene utilizzata la libreria `pydantic` per definire un modello di dati strutturato (`User`) con campi specifici e descrizioni. Questo modello serve a strutturare le risposte in un formato coerente. Successivamente, si utilizza `PydanticOutputParser` di LangChain per creare un parser che formatta le risposte del modello LLM in conformità con il modello `User`.

### Assignment per i Corsisti
- Scrivi il codice per definire un modello Pydantic personalizzato e crea un `PydanticOutputParser` basato su di esso.
- Stampa le istruzioni di formattazione del parser per capire come il modello LLM dovrebbe strutturare l'output.
- Come esercizio, prova a espandere il modello `User` con altri campi (es. `age`, `role`) e osserva come le istruzioni di formattazione cambiano.


In [9]:
from pydantic import BaseModel, Field, validator

class User(BaseModel):
    pass
    

from langchain.output_parsers import PydanticOutputParser



### Creazione di un Prompt con Istruzioni di Formattazione Personalizzate
In questa cella, viene creato un `PromptTemplate` che utilizza le istruzioni di formattazione generate dal `PydanticOutputParser`. Il template chiede di analizzare un testo e include le istruzioni di formattazione come parte del prompt. Questo consente di garantire che le risposte del modello LLM siano strutturate in conformità con il modello Pydantic specificato.

### Assignment per i Corsisti
- Scrivi il codice per creare un `PromptTemplate` che includa le istruzioni di formattazione del parser.
- Verifica che il template sia configurato correttamente e pronto per essere utilizzato in una catena di esecuzione con il modello LLM.
- Come esercizio, prova a creare un prompt simile per altri modelli Pydantic e osserva come la formattazione influisce sull'output generato.


In [ ]:
# 

In [12]:
# Use a structured description for the user

In [13]:
# Use a more verbose description

In [14]:
# Use a more verbose description and create a chain to serialize a JSON object
from langchain_core.output_parsers import JsonOutputParser